In [131]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader
from data import load_traindata
device = 'mps' if torch.backends.mps.is_available() else ('cuda' if torch.cuda.is_available() else 'cpu')
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import random_split

In [132]:
batch_size = 32
num_subclasses = 500
learning_rate = 0.0001
epochs = 1000
test_epochs = 10
seq_size = 500

In [133]:
# Encoder function (One-hot encoding)
def one_hot_encode(labels, unique_labels):
    # Create a mapping from unique labels to indices
    label_to_index = {label.item(): idx for idx, label in enumerate(unique_labels)}
    
    # Convert the labels to indices based on the mapping
    indices = torch.tensor([label_to_index[label.item()] for label in labels])
    
    # Create the one-hot encoded tensor
    return torch.eye(len(unique_labels))[indices]

# Decoder function (Converts one-hot back to original labels)
def one_hot_decode(one_hot, unique_labels):
    # Get the index of the '1' in the one-hot vector
    index = torch.argmax(one_hot)
    return torch.tensor(unique_labels[index])

In [134]:
#load dataset
X,Y = load_traindata(num_subclasses)
X = np.array(X)
X = torch.from_numpy(X)
X = X.view(num_subclasses, 5000/seq_size,seq_size,12) #reshape after split
X = X.view(num_subclasses*(5000/seq_size),seq_size,12)
unique_labels = Y  # Unique labels are just the 500 unique values in Y
Y = one_hot_encode(Y, unique_labels)  # One-hot encode Y
Y = Y.unsqueeze(1).repeat(1, 5000/seq_size, 1).view(-1, num_subclasses)


TypeError: view(): argument 'size' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got float"

In [ ]:
class ECGDataset(Dataset):
    def __init__(self,X,Y):  
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return X[idx,:,0], Y[idx]

In [ ]:
def plotWave(X):
   # Create a common time axis for all leads
    time = torch.arange(0, X.size(0))

    # Create subplots (12 rows, 1 column)
    fig, axes = plt.subplots(12, 1, figsize=(10, 24), sharex=True, sharey=True)
    fig.suptitle('12 Lead ECG Report', fontsize=16)

    # Plot each lead in a separate subplot
    for i in range(12):
        axes[i].plot(time, X[:, i].numpy(), color='b')
        axes[i].set_ylabel(f'Lead {i+1}')
        axes[i].grid(True)

    # Set common X-axis label for time
    axes[11].set_xlabel('Time')

    # Adjust layout
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [ ]:
dataset = ECGDataset(X,Y)
train_size = int(0.9 * len(dataset)) 
test_size = len(dataset) - train_size  
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        # Define the TransformerEncoderLayer blocks
        self.block1 = nn.TransformerEncoderLayer(d_model=100, nhead=10)
        self.block2 = nn.TransformerEncoderLayer(d_model=100, nhead=10)
        self.block3 = nn.TransformerEncoderLayer(d_model=100, nhead=10)
        # Layer normalization
        self.xnorm1 = nn.LayerNorm(100)
        self.fc1 = nn.Linear(100, 128)
        self.xnorm2 = nn.LayerNorm(128)
        self.fc2 = nn.Linear(128, num_subclasses)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = F.relu(self.fc1(x))
        x = self.xnorm2(x)
        x = self.fc2(x)
        x = torch.softmax(x)
        return x


In [ ]:
model = Model()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

1.435528 M parameters


In [ ]:
losses = []

In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
for epoch in range(epochs):
    for x, y in dataloader:
        x,y = x.to(torch.float32), y.to(torch.float32)
        x,y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = m(x)
        loss = F.binary_cross_entropy(out, y)
        loss.backward()
        optimizer.step()
    losses.append(loss.item())
    if epoch % test_epochs == 0:
        print(f'Epoch {epoch}, loss: {loss.item()}')

Python(56730) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(56731) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(56735) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(56736) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Applications/Xcode.app/Contents/Developer/Library/Frameworks/Python3.framewor

RuntimeError: DataLoader worker (pid(s) 56735, 56736) exited unexpectedly

In [ ]:
import torch
# Assuming the model and test_loader have been defined
# model.eval() switches the model to evaluation mode
model.eval()
# Initialize variables to track correct predictions and total predictions
correct = 0
total = 0
# Disable gradient computation during evaluation
with torch.no_grad():
    # Loop over the test dataset
    for data, labels in test_loader:
        # Move data to the appropriate device (if using CUDA)
        data,labels = data.to(torch.float32), labels.to(torch.float32)
        data, labels = data.to(device), labels.to(device)
        # Get model predictions
        outputs = model(data)
        _, true_labels = torch.max(labels, 1)
        # Get the predicted class by taking the argmax (class with highest score)
        _, predicted = torch.max(outputs, 1)
        # Update the total number of samples and correct predictions
        total += labels.size(0)
        correct += (predicted == true_labels).sum().item()

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Accuracy on test dataset: {accuracy:.2f}%')


Accuracy on test dataset: 1.20%
